In [ ]:
import pandas as pd
import numpy as np
from numpy.linalg import inv
from scipy.special import betainc, beta
import scipy
from random import seed
from numpy import dot, ones

In [ ]:
def mu_g(N, mu, sigma):
    p1 = dot(ones(N), dot(inv(sigma), mu))
    p2 = dot(ones(N), dot(inv(sigma), ones(N)))
    return p1/p2

def w_g(N, sigma):
    p1 = (dot(inv(sigma), ones(N)))
    p2 = (dot(ones(N), dot(inv(sigma), ones(N))))
    return p1/p2

def w_z(N, mu, sigma):
    mug = mu_g(N, mu, sigma)
    br = mu - ones(N)*mug
    return dot(inv(sigma), br)

def w_star(N, mu, sigma, gamma):
    return w_g(N, sigma) + (1/gamma)*w_z(N, mu, sigma)

In [ ]:
def psi_sq(N, mu, sigma):
    mug = mu_g(N, mu, sigma)
    br = mu - ones(N)*mug
    return dot(br, dot(inv(sigma), br))

def psi_asq(N, h, mu, sigma):
    psq = psi_sq(N, mu, sigma)
    p1 = ((h - N - 1)*psq - (N - 1))/h
    n1 = psq**((N-1)/2)
    n2 = (1 + psq)**(-(h-2)/2)
    a = (N - 1)/2 ; b = (h - N + 1)/2
    x = psq/(1+psq)
    b = beta(a, b)*betainc(a,b,x)
    p2 = (2*n1*n2)/(h*b)
    return p1 + p2

def k_func(N, h):
    return ((h - N)*(h - N - 3))/(h*(h-2))

def w_qt(N, h, mu, sigma, gamma):
    k = k_func(N, h)
    pasq = psi_asq(N, h, mu, sigma)
    c1 = k*pasq; c2 = pasq + (N - 1)/h
    ct = c1/c2
    wg = w_g(N, sigma) ; wz = w_z(N, mu, sigma)
    return wg + (ct/gamma)*wz

For the implementable rules:

In [ ]:
def calc_sim_perf(
        returns_data: pd.DataFrame,
        h: int,
        gamma: int
):
    # column names for forthcoming dataframe
    cols = ["t",
            "Implementable",
            "Plug-in"
            ]
    # set blank lists for returns
    rets = []
    track = 0
    for t in range(h, len(returns_data)):
        # select which part of the file is being used for the weight calculation
        est = returns_data[track:t - 1]
        # filter out assets with at least 120 NaNs in the estimation window
        valid_assets = est.columns[est.isna().sum() < h]
        est = est[valid_assets]
        N_temp = len(valid_assets)
        # determine which line is being used to calculate returns
        time_t_data = returns_data.loc[t, valid_assets].values
        # set mu and sigma estimates
        mu_hat = est.mean()
        sigma_hat = est.cov()
        # calculate all of the portfolios
        w_imp_opt = w_qt(N_temp, h, mu_hat, sigma_hat, gamma)
        w_pt = w_star(N_temp, mu_hat, sigma_hat, gamma)
        row = []; row.append(t)
        port_list = [w_imp_opt, w_pt]
        for portfolio in port_list:
            # calculate the returns on the data
            row.append(dot(portfolio, time_t_data))
        rets.append(row)
    return pd.DataFrame(rets, columns=cols)

For the optimal and naive rules:

In [ ]:
def calc_sim_perf_alt(
        returns_data: pd.DataFrame,
        gamma: int
):
    # set the number of assets
    N = len(returns_data.columns)
    # column names for forthcoming dataframe
    cols = ["t", "Optimal", "Naive"]
    # set blank lists for returns
    rets = []
    # take the true mean and covariance (i.e. over entire history)
    tmean = returns_data.mean()
    tsigma = returns_data.cov()
    # calculate the optimal portfolios
    w_opt = w_star(N, tmean, tsigma, gamma)
    
    for t in range(len(returns_data)):
        # determine which line is being used to calculate returns
        time_t_data = returns_data.iloc[t]
        
        # Filter out assets with NaNs in the current row
        valid_assets = time_t_data.dropna().index
        time_t_data = time_t_data[valid_assets].values
        
        # Calculate the returns for w_opt and w_naive for each row
        valid_asset_indices = [returns_data.columns.get_loc(asset) for asset in valid_assets]
        w_opt_valid = w_opt[valid_asset_indices]  # Filter w_opt for valid assets
        w_naive_valid = np.ones(len(valid_assets)) / len(valid_assets)  # Naive weights already adjusted for valid assets
        row = [t]
        row.append(np.dot(w_opt_valid, time_t_data))  # Return for optimal portfolio
        row.append(np.dot(w_naive_valid, time_t_data))  # Return for naive portfolio
        rets.append(row)
    return pd.DataFrame(rets, columns=cols)

In [ ]:
def find_sharpe(mu, sigma):
    # just return the ratio
    return mu/sigma

def find_cer(mu, sigma, gamma):
    # return the certainty equivalent return
    return mu - (gamma/2)*sigma**2

Here, only a few of the files have missing values. They are handled accordingly.

(Sourced from Ken French's website)

In [ ]:
# read in the csv with the 5 portfolio data
fiveinddata = pd.read_csv("/Users/senadkokic/Desktop/MRP/Data/5_industry.csv", sep = "\t")
# rename the first column
fiveinddata = fiveinddata.rename(columns = {"Unnamed: 0":"Date"})
# consider column 2 onwards
fiveinddata = fiveinddata[fiveinddata.columns[1:]]

# read in the csv with the 10 portfolio data
tenmomdata = pd.read_csv("/Users/senadkokic/Desktop/MRP/Data/10_Momentum_Portfolios_Monthly.csv", sep = "\t")
# rename the first column
tenmomdata = tenmomdata.rename(columns = {"Unnamed: 0":"Date"})
tenmomdata = tenmomdata[tenmomdata.columns[1:]]

# read in the csv with the 25 portfolio data
fama_tt_data = pd.read_csv("/Users/senadkokic/Desktop/MRP/Data/25_Size_Book_To_Market_Adj.csv", sep = "\t")
# rename the first column
fama_tt_data = fama_tt_data.rename(columns = {"Unnamed: 0":"Date"})
# consider columns 2 onwards
fama_tt_data = fama_tt_data[fama_tt_data.columns[1:]]
fama_tt_data.replace(-99.99, np.nan, inplace=True)

# load the 32 data
thrtwo_dat = pd.read_csv("/Users/senadkokic/Desktop/MRP/Data/32_op.csv",sep = "\t")
# rename the first column
thrtwo_dat = thrtwo_dat.rename(columns = {"Unnamed: 0":"Date"})
# consider columns 2 onwards
thrtwo_dat = thrtwo_dat[thrtwo_dat.columns[1:]]

# read in the csv with the 100 portfolio data
fornine = pd.read_csv("/Users/senadkokic/Desktop/MRP/Data/49.csv", sep = "\t")
# rename the first column
fornine = fornine.rename(columns = {"Unnamed: 0":"Date"})
# consider column 2 onwards
fornine = fornine[fornine.columns[1:]]
fornine.replace(-99.99, np.nan, inplace=True)

In [ ]:
np.random.seed(32324)
l = ["fiveinddata", "tenmomdata", "fama_tt_data","thrtwo_dat","fornine"]
q = 0
for dataset in [fiveinddata, tenmomdata, fama_tt_data,thrtwo_dat,fornine]:
    dataset=dataset/100
    res = calc_sim_perf(dataset, h = 120, gamma = 3)
    means = res.mean()[1:] ; sds = res.std()[1:]
    #print("Dataset:", l[q], "\n", round(find_cer(means, sds, gamma = 3),4))
    print("Dataset:", l[q], "\n", round(find_sharpe(means, sds), 4))
    q += 1

In [ ]:
l = ["fiveinddata", "tenmomdata", "fama_tt_data","thrtwo_dat","fornine"]
q = 0
for dataset in [fiveinddata, tenmomdata, fama_tt_data,thrtwo_dat,fornine]:
    dataset=dataset/100
    res = calc_sim_perf_alt(dataset, gamma = 3)
    means = res.mean()[1:] ; sds = res.std()[1:]
    #print("Dataset:", l[q], "\n", round(find_cer(means, sds, gamma = 3),4))
    print("Dataset:", l[q], "\n", round(find_sharpe(means, sds), 4))
    q += 1